In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaForSequenceClassification
from transformers import AdamW
import random
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.spatial.distance import cosine
from sim_utils import load_examples, Inputexample, CustomTextDataset, freeze_layers, train, test
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig, AutoModel, AutoTokenizer

os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [2]:
!nvidia-smi

Sun May  1 15:38:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:84:00.0 Off |                  N/A |
| 24%   27C    P8    11W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [3]:
N = 10

data = []
labels = []

train_samples = []
train_labels = []

valid_samples = []
valid_labels = []

test_samples = []
test_labels = []

embed_dim = 768
batch_size = 16 
lr=2e-3  # you can adjust 
temp = 0.3  # you can adjust 
lamda = 0.01  # you can adjust  
skip_time = 0 # the number of time that yi not equal to yj in supervised contrastive loss equation 

data_names = ['CLINC150','BANKING77','HWU64'] 
model_names = ['roberta-base','bert-base-uncased']
shot_names = ['train_5','train_10']
lines = ['test acc']

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

In [4]:
!nvidia-smi

Sun May  1 15:38:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:84:00.0 Off |                  N/A |
| 24%   27C    P8    11W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:85:00.0 Off |                  N/A |
| 22%   

In [5]:
for shot_name in shot_names:
    for data_name in data_names:

        path_shot = f'../../../{data_name}/{shot_name}/'

        valid_path = f'../../../{data_name}/valid/'
        test_path = f'../../../{data_name}/test/'


        # load data
        train_samples = load_examples(path_shot)
        valid_samples = load_examples(valid_path)
        test_samples = load_examples(test_path)


        print("===== small train set ====")
        
        data = []
        labels = []

        for i in range(len(train_samples)):
            data.append(train_samples[i].text)
            labels.append(train_samples[i].label)


        train_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)



        print("===== validation set ====")

        data = []
        labels = []

        for i in range(len(valid_samples)):
            data.append(valid_samples[i].text)
            labels.append(valid_samples[i].label)

        valid_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        valid_loader = DataLoader(valid_data,batch_size=batch_size,shuffle=True)



        print("===== test set ====")

        data = []
        labels = []

        for i in range(len(test_samples)):
            data.append(test_samples[i].text)
            labels.append(test_samples[i].label)

        test_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=False)
        test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)



         # got the number of unique classes from dataset
        num_class = len(np.unique(np.array(labels)))

         # get text label of uniqure classes
        unique_label = np.unique(np.array(labels))

         # map text label to index classes
        label_maps = {unique_label[i]: i for i in range(len(unique_label))}

        print("label_maps :",label_maps)
        print("num_class:",num_class)


        
        for model_name in model_names:



            exp_name = f'{model_name}_lr={lr}_t={temp}_{data_name}_{shot_name}'
            direct_name = f"{model_name}"

            print("direct_name :",direct_name)
            tokenizer = AutoTokenizer.from_pretrained(direct_name)
            config = AutoConfig.from_pretrained(direct_name)
            config.num_labels = num_class
            simcse = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)

            simcse = freeze_layers(simcse,freeze_layers_count=12)
            optimizer= AdamW(simcse.parameters(), lr=lr)
            simcse = simcse.to(device)

            train_log, valid_log = train(exp_name,simcse,device,label_maps,optimizer,train_loader,valid_loader,train_data,valid_data,tokenizer,epochs=30)




            PATH = f'../../../fewshot_models/{exp_name}.pth'
            best_model = AutoModelForSequenceClassification.from_pretrained(direct_name,config=config)
            # Model class must be defined somewhere
            best_model.load_state_dict(torch.load(PATH))
            best_model = best_model.to(device)


            test_acc = test(best_model,device,label_maps,test_loader,len(test_data),tokenizer)

            test_acc = 100 * test_acc
            res = f'shot:{shot_name}_data_name:{data_name}_model:{model_name}_test_acc:{str(test_acc)}'
            lines.append(res)


with open(f'result_baseline.txt', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')

===== small train set ====
Train on Cross Entropy loss
len of dataset : 750
===== validation set ====
Train on Cross Entropy loss
len of dataset : 3000
===== test set ====
Train on Cross Entropy loss
len of dataset : 4500
label_maps : {'accept_reservations': 0, 'account_blocked': 1, 'alarm': 2, 'application_status': 3, 'apr': 4, 'are_you_a_bot': 5, 'balance': 6, 'bill_balance': 7, 'bill_due': 8, 'book_flight': 9, 'book_hotel': 10, 'calculator': 11, 'calendar': 12, 'calendar_update': 13, 'calories': 14, 'cancel': 15, 'cancel_reservation': 16, 'car_rental': 17, 'card_declined': 18, 'carry_on': 19, 'change_accent': 20, 'change_ai_name': 21, 'change_language': 22, 'change_speed': 23, 'change_user_name': 24, 'change_volume': 25, 'confirm_reservation': 26, 'cook_time': 27, 'credit_limit': 28, 'credit_limit_change': 29, 'credit_score': 30, 'current_location': 31, 'damaged_card': 32, 'date': 33, 'definition': 34, 'direct_deposit': 35, 'directions': 36, 'distance': 37, 'do_you_have_pets': 38, '

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

correct : 685
total : 1076
direct_name : bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

classifier.weight
classifier.bias
======  Epoch 1 ====== 
 Training Loss: 0.2955379605293274, 		 Training acc: 0.01718750037252903
train correct :  tensor(11, device='cuda:3')
train total : 640
 Validation Loss: 0.2700623643885758, 		 Validation acc: 0.026022303849458694
valid correct :  tensor(28, device='cuda:3')
valid total : 1076
Validation Loss Decreased(inf--->0.270062) 	 Saving The Model
======  Epoch 2 ====== 
 Training Loss: 0.2707455702126026, 		 Training acc: 0.02812500111758709
train correct :  tensor(18, device='cuda:3')
train total : 640
 Validation Loss: 0.26352239740825495, 		 Validation acc: 0.0511152409017086
valid correct :  tensor(55, device='cuda:3')
valid total : 1076
Validation Loss Decreased(0.270062--->0.263522) 	 Saving The Model
======  Epoch 3 ====== 
 Training Loss: 0.26060586869716645, 		 Training acc: 0.05312500149011612
train correct :  tensor(34, device='cuda:3')
train total : 640
 Validation Loss: 0.25279310711254416, 		 Validation acc: 0.0613382905721

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

correct : 296
total : 1076
